## Memanggil library yang diperlukan

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

## Membuka file (total 450 files)

In [ ]:
# Ambil dokumen dari folder (450 file, agar pembentukan inverted index tidak memakan waktu lama)
raw_doc_collection = []
proc_doc_collection = []
path = '../dataset-retrieval/talk.politics.misc.450/'
files = os.listdir(path)
for file in files:
    f = open(path+file)
    raw_doc_collection.append(f.read())

## Membuat vocabulary list

In [ ]:
# Pembentukan vocabulary menggunakan tokenizer NLTK
# hanya token yang tidak mengandung angka dan simbol 
# serta panjangnya lebih dari 3 karater yang diambil
def get_vocabulary(raw_doc_collection) :
    s = []
    for i in range(len(raw_doc_collection)) :
        l = [token.lower() for token in word_tokenize(raw_doc_collection[i])]
        
        # stemming
        start_stemming = time.time()
        snow_stemmer = SnowballStemmer(language='english')
        l = [snow_stemmer.stem(x) for x in l]
        stop_stemming = time.time()
        
        proc_doc_collection.append([])
        for j in range(len(l)) :
            if l[j].isalpha() and len(l[j]) > 3  :
                s.append(l[j])
                proc_doc_collection[i].append(l[j])
    vocabulary = list(set(s))
    stemming_runtime = stop_stemming - start_stemming
    return vocabulary, stemming_runtime

## preprocessing

In [ ]:
# Start 1 timer
start1 = time.time()

# Preprocessing
lexicon, stemming_runtime = get_vocabulary(raw_doc_collection)

# Stop words removal
stop_words = set(stopwords.words('english')) 
lexicon = [w for w in lexicon if not w in stop_words]

lexicon = list(set(lexicon))
lexicon.sort()

# print(lexicon)

# Stop 1 timer
stop1 = time.time()

print("\nStemming runtime:",stemming_runtime, " seconds")
print("\nVocabulary built in ", stop1-start1, " seconds")
print("\nVocabulary size :", len(lexicon))
print(lexicon)


Stemming runtime: 0.0027794837951660156  seconds

Vocabulary built in  5.3980066776275635  seconds

Vocabulary size : 7236
['aaron', 'abandon', 'abandond', 'abba', 'abbasov', 'abbey', 'abdul', 'abhor', 'abhorr', 'abid', 'abil', 'abilen', 'abod', 'abolish', 'abomin', 'aboritionist', 'abort', 'aborte', 'abov', 'abraham', 'abridg', 'abroad', 'abrog', 'abrupt', 'absenc', 'absolut', 'absolv', 'absorb', 'abstract', 'absurd', 'abund', 'abus', 'abyss', 'academ', 'academi', 'academia', 'acced', 'acceler', 'accept', 'access', 'accid', 'accommod', 'accomod', 'accomplish', 'accord', 'account', 'accret', 'accuar', 'accumul', 'accur', 'accuraci', 'accus', 'accuss', 'achi', 'achiev', 'achill', 'acid', 'acknowledg', 'aclu', 'acquir', 'acquisit', 'acquit', 'acquitt', 'across', 'action', 'actionm', 'activ', 'activist', 'actual', 'actuali', 'actuari', 'acun', 'acunerbb', 'adam', 'adapt', 'addict', 'addit', 'address', 'adequ', 'adher', 'adject', 'adjust', 'admin', 'administ', 'administr', 'admir', 'admir

## Membuat inverted index

In [ ]:
# Start 2 timer
start2 = time.time()

# Pembentukan inverted index
inverted_index = []
for i in range(len(lexicon)) :
    inverted_index.append([])
    for j in range(len(proc_doc_collection)) :
        if lexicon[i] in proc_doc_collection[j] :
            inverted_index[i].append(j)

# print(inverted_index)    

# Stop 2 timer
stop2 = time.time()
print("\nInverted index built in ", stop2-start2, " seconds")


Inverted index built in  21.744018077850342  seconds


## Menghitung term frequency

In [ ]:
term_frequency = [0 for i in range(len(lexicon))]
for i in range(len(lexicon)) :
    term_frequency[i] = len(inverted_index[i])
# print(term_frequency)

## Binary search, untuk searching kata di vocabulary

In [ ]:
# binary search to find id of word in vocabulary/lexicon
def get_word_id(word) :
    l,r = 0,len(lexicon)-1
    while(l<=r) :
        m = (l+r)//2
        if lexicon[m]==word : 
            return m
        elif lexicon[m]<word :
            l=m+1
        else :
            r=m-1
    return -1

## Intersection antara 2 postings

In [ ]:
# efficient two pointer approach for the intersection of 2 postings list , as mentioned in the book
def intersection_of_postings(l1 , l2) :
    answer=[]
    p1,p2=0,0
    while p1!=len(l1) and p2!=len(l2) :
        if l1[p1]==l2[p2] : 
            answer.append(l1[p1])
            p1+=1
            p2+=1
        elif l1[p1]<l2[p2] :
            p1+=1
        else :
            p2+=1
    return answer

## Intersection dari beberapa keywords

In [ ]:
# efficient approach for the intersection of n postings list , as mentioned in the book
def n_intersection(words_list) :
    #id1 stores id of words whose postings are to be intersected
    id1,answer=[],[]
    for word in words_list : id1.append(get_word_id(word))
    tuples_list=[]
    for i in range(len(id1)) : 
        tuples_list.append((term_frequency[id1[i]],id1[i]))
    #sort postings list according to term frequency for best order of query processing
    tuples_list.sort()
    id0 = tuples_list[0][1]
    if id0!=-1 : answer = inverted_index[id0]
    for i in range(1,len(tuples_list)) :
        word_id = tuples_list[i][1]
        if word_id == -1 : continue
        answer = intersection_of_postings(answer , inverted_index[word_id])
    return answer    

In [ ]:
## union (OR)
def union_of_postings(l1,l2):
#     answer=np.unique(l1+l2)
    answer=[]
    p1,p2=0,0
    while p1!=len(l1):
        if l1[p1] not in answer:
            answer.append(l1[p1])
        p1+=1
    
    while p2!=len(l2):
        if l2[p2] not in answer:
            answer.append(l2[p2])
        p2+=1
    return answer

def n_union(words_list):
    id1,answer=[],[]
    for word in words_list : id1.append(get_word_id(word))
    tuples_list=[]
    for i in range(len(id1)) : 
        tuples_list.append((term_frequency[id1[i]],id1[i]))
    #sort postings list according to term frequency for best order of query processing
    tuples_list.sort()
    id0 = tuples_list[0][1]
    if id0!=-1 : answer = inverted_index[id0]
    for i in range(1,len(tuples_list)) :
        word_id = tuples_list[i][1]
        if word_id == -1 : continue
        answer = union_of_postings(answer , inverted_index[word_id])
    return answer

In [ ]:
## negation
## all documents - union
def n_negation(words_list):
    id1,answer=[],[]
    for word in words_list : id1.append(get_word_id(word))
    tuples_list=[]
    for i in range(len(id1)) : 
        tuples_list.append((term_frequency[id1[i]],id1[i]))
    #sort postings list according to term frequency for best order of query processing
    tuples_list.sort()
    id0 = tuples_list[0][1]
    if id0!=-1 : answer = inverted_index[id0]
    for i in range(1,len(tuples_list)) :
        word_id = tuples_list[i][1]
        if word_id == -1 : continue
        answer = union_of_postings(answer , inverted_index[word_id])
    
    union_docs = []
    for doc_id in answer:
        union_docs.append(raw_doc_collection[doc_id])
    
    negation_docs = []
    for doc_id in raw_doc_collection:
        if doc_id not in union_docs:
            negation_docs.append(doc_id)
    
    return negation_docs

## input dari user

In [ ]:
#take input queries from user
print("Enter type of query -> 1,2,3:")
print("[1.] AND ")
print("[2.] OR ")
print("[3.] NEGATION ")
type = int(input())
n = int(input('Enter number of terms to take : '))
if n<1:
    print("enter valid inputs only.")
l=[]
for i in range(n):
    print('Enter word number ',i+1)
    word= input()
    snow_stemmer = SnowballStemmer(language='english')
    word = snow_stemmer.stem(word)
    l.append(word)
# Start 3 timer
start3 = time.time()

if type==1:
    doc_id = n_intersection(l)
    print("The relevant documents are (", len(doc_id), ") :")
    for id1 in doc_id :
        print(raw_doc_collection[id1])
elif type==2:
    doc_id = n_union(l)
    print("The relevant documents are (", len(doc_id), ") :")
    for id1 in doc_id :
        print(raw_doc_collection[id1])
elif type==3:
    docs = n_negation(l)
    print("The relevant documents are (", len(docs), ") :")
    for doc in docs:
        print(doc)
else:
    print("not in option")

# Stop 3 timer
stop3 = time.time()
print("\nQuerying in ", stop3-start3, " seconds")

Enter type of query -> 1,2,3:
[1.] AND 
[2.] OR 
[3.] NEGATION 
2
Enter number of terms to take : 3
Enter word number  1
people
Enter word number  2
support
Enter word number  3
find
The relevant documents are ( 264 ) :
In article <1993Apr14.195912.16613@grace.rt.cs.boeing.com> rwojcik@atc.boeing.com (Richard Wojcik) writes:
>In article 734629856@misty, john@anasazi.com (John R. Moore) writes:
>>papresco@undergrad.math.uwaterloo.ca (Paul Prescod) writes:
>>]I'm not.  I'm in Canada.  We have far fewer shootings like this.  We have
>>]had, I believe, one mass murder in the last twenty years.
>>]I'm not going to say we don't have our gun problems.  But we do have the
>>]world's largest undefended boarder with one of the most gun-happy countries 
>>]in the world.  I think Canada illustrates that gun control does have an 
>>]effect.  In fact, it's suprising that there is any difference considering
>>]how easy it is to smuggle a gun from the U.S.
>>Yes, it's amazing, isn't it. In fact, it sh